In [2]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier
import joblib

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


#read the csv file
df = pd.read_csv('data/stat_acc_V3_new.csv', sep=';')


### Répartition des données/apprentissage/test

In [3]:
# Séparation de la base de donnée en base de test et d'apprentissage
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

#initialisation des variables X et Y (à modifier avec les jeux de données d'apprentissage)
X = train_set.drop(['descr_grav'], axis=1)
Y = train_set['descr_grav']

#### Holdout Répartition

In [4]:
# Répétition 5 fois de holdout split avec prédiction et évaluation
for i in range(5):
    # Division des données en ensembles d'apprentissage et de tests
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=i)
    
    # Création du modèle KNN
    knn = KNeighborsClassifier(n_neighbors=5)
    
    # Apprentissage du modèle KNN
    knn.fit(X_train, y_train)
    
    # Prédiction des données de test
    y_pred = knn.predict(X_test)
    
    # Evaluation de la prédiction
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)


Accuracy: 0.36722396673173213
Accuracy: 0.37053381991003986
Accuracy: 0.35984044810319954
Accuracy: 0.36306543325129426
Accuracy: 0.3610286005261818


#### Leave One out Répartition

In [11]:
from sklearn.model_selection import LeaveOneOut

# Création de l'objet leave one out
loo = LeaveOneOut()

# Création du compteur
count = 0

# Execution de leave one out
for train_index, test_index in loo.split(X):
    # Sortie du programme
    count += 1  
    if count >= 100:
        break 
    
    # Répartition 
    #print("%s %s" % (train_index, test_index))
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    
    # Apprentissage du modèle KNN
    knn.fit(X_train, y_train)
    
    # Prédiction des données de test
    y_pred = knn.predict(X_test)
    
    # Evaluation de la prédiction
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)

Accuracy: 0.0
Accuracy: 1.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 1.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 1.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 1.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 1.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 1.0
Accuracy: 0.0
Accuracy: 1.0
Accuracy: 1.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 1.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 1.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 1.0
Accuracy: 1.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 1.0
Accuracy: 0.0
Accuracy: 1.0
Accuracy: 1.0
Accuracy: 0.0
Accuracy: 0.0
Accuracy: 1.0
Accuracy: 1.0
Accuracy: 0.0
Accuracy: 1.0
Accuracy: 0.0
Accura

### Classification avec SVM

In [ ]:
def SVM_class(X,Y,old_model=None):
    #création d'un classifieur SVM
    svm = SVC()
    svm_params = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
    svm_grid = GridSearchCV(svm, svm_params, cv=5)
    svm_grid.fit(X, Y)
    best_svm = svm_grid.best_estimator_

    #enregistrement du modèle avec les meilleurs paramètres
    #comparaison avec l'ancien modèle pour ne pas le remplacer si moins performant
    if old_model == None:
        joblib.dump(best_svm, 'models/svm_model.pkl')
    else:
        if old_model.score(X,Y) < best_svm.score(X,Y):
            joblib.dump(best_svm, 'models/svm_model.pkl')

    return best_svm


### Classification avec Random Forest

In [ ]:
def RF_class(X,Y,old_model=None):
    #création d'un classifieur Random Forest
    rf = RandomForestClassifier()
    rf_params = {'n_estimators':[10, 100, 1000], 'max_depth':[None, 10, 100]}
    rf_grid = GridSearchCV(rf, rf_params, cv=5)
    rf_grid.fit(X, Y)
    best_rf = rf_grid.best_estimator_

    #enregistrement du modèle avec les meilleurs paramètres
    #comparaison avec l'ancien modèle pour ne pas le remplacer si moins performant
    if old_model == None:
        joblib.dump(best_rf, 'models/rf_model.pkl')
    else:
        if old_model.score(X,Y) < best_rf.score(X,Y):
            joblib.dump(best_rf, 'models/rf_model.pkl')


    return best_rf


### Classification avec MLP

In [ ]:
def MLP_class(X,Y,old_model=None):
    #création d'un classifieur MLP
    mlp = MLPClassifier()
    mlp_params = {'hidden_layer_sizes':[(10,), (100,), (1000,)], 'activation':['relu', 'tanh', 'logistic']}
    mlp_grid = GridSearchCV(mlp, mlp_params, cv=5)
    mlp_grid.fit(X, Y)
    best_mlp = mlp_grid.best_estimator_

    #enregistrement du modèle avec les meilleurs paramètres
    #comparaison avec l'ancien modèle pour ne pas le remplacer si moins performant
    if old_model == None:
        joblib.dump(best_mlp, 'models/mlp_model.pkl')
    else:
        if old_model.score(X,Y) < best_mlp.score(X,Y):
            joblib.dump(best_mlp, 'models/mlp_model.pkl')

    return best_mlp

### Fusion classifieurs

In [ ]:
#fusion des classifieurs

#vote majoritaire
voting_clf_maj = VotingClassifier(estimators=[('svm', best_svm), ('rf', best_rf), ('mlp', best_mlp)], voting='hard')
voting_clf_maj.fit(X, Y)

#vote pondéré
voting_clf_pond = VotingClassifier(estimators=[('svm', best_svm), ('rf', best_rf), ('mlp', best_mlp)], voting='soft')
voting_clf_pond.fit(X, Y)

#prediction sur un nouveau jeu de données (à modifier avec les jeux de données de test)
X_pred = X_test

#prediction vote majoritaire
Y_pred_maj = voting_clf_maj.predict(X_test)

#prediction vote pondéré
Y_pred_pond = voting_clf_pond.predict(X_test)

NameError: name 'best_svm' is not defined